In [ ]:
#importing pckages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
import statsmodels.api as sm  
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
import os
print(os.listdir("../input/housesalesprediction"))

# Data Loading, Processing and Visualization

In [ ]:
#loading data
df = pd.read_csv("..//input//housesalesprediction//kc_house_data.csv")

In [ ]:
#Get the information about data
df.info()

In [ ]:
#Find out the statstics about data
df.describe()

In [ ]:
#Get the sample view from top
df.head()

In [ ]:
#Get the sample view from bottom
df.tail()

In [ ]:
# Looking Missing Values
print(df.isnull().any())

In [ ]:
#Data is pretty clean. Now dropping two columns which are going to used in prediction. Date and ID
df = df.drop(['id', 'date'],axis=1)

In [ ]:
# Let's visualize the data 
#sns.pairplot(df)

In [ ]:
sns.pairplot(df[['sqft_lot','sqft_above','price','sqft_living','bedrooms']], palette='afmhot',height=1.6)

In [ ]:
#Rescaling the features
#defining a normalisation function 
def normalize (x): 
    return ( (x-np.mean(x))/ (max(x) - min(x)))
                                            
                                              
# applying normalize ( ) to all columns 
df = df.apply(normalize)

# Splitting Data , train and test sets

In [ ]:
df.columns

In [ ]:
# Putting feature variable to X
X=df[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15']]
# Putting feature variable to y
y=df['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7 ,test_size = 0.3, random_state=100)

# Running RFE

In [ ]:
# Running RFE with the output number of the variable 
lm = LinearRegression()
rfe = RFE(lm, 9)             # running RFE
rfe = rfe.fit(X_train, y_train)
print(rfe.support_)           # Printing the boolean results
print(rfe.ranking_)  

In [ ]:
col = X_train.columns[rfe.support_]

In [ ]:
col

# Building Model

In [ ]:
# Creating X_test dataframe with RFE selected variables
X_train_rfe = X_train[col]

In [ ]:
X_train_rfe.columns

In [ ]:
# Adding a constant variable 
X_train_rfe = sm.add_constant(X_train_rfe)

In [ ]:
lm = sm.OLS(y_train,X_train_rfe).fit()   # Running the linear model

In [ ]:
#Let's see the summary of our linear model
print(lm.summary())

In [ ]:
vif=pd.DataFrame()

In [ ]:
vif['VIF'] = [variance_inflation_factor(X_train_rfe.values, i) for i in range(X_train_rfe.shape[1])]

In [ ]:
 vif["variables"] = X_train_rfe.columns

In [ ]:
vif

# Dropping the Variable and Updating the Model

In [ ]:
# Dropping highly correlated variables and insignificant variables
X_train_new = X_train_rfe.drop('sqft_living', 1)

In [ ]:
# Create a second fitted model
lm_2 = sm.OLS(y_train,X_train_new).fit()

In [ ]:
#Let's see the summary of our second linear model
print(lm_2.summary())

In [ ]:
vif=pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X_train_new.values, i) for i in range(X_train_new.shape[1])]
vif["variables"] = X_train_new.columns
vif

In [ ]:
# Now we got all the VIF under 5 and p value of existing variables is 0.000
# R-squared is Approx 69 %  there is no very minute differenece between R-squared and Adjusted R-Squraed. 
# We can say these are the important features which are more useful to predict the House Price.

# Prediction using Model 2

In [ ]:
# Adding  constant variable to test dataframe
X_test_m2 = sm.add_constant(X_test)

In [ ]:
# Creating X_test_m6 dataframe by dropping variables from X_test_m6
X_test_m2 = X_test_m2.drop(['sqft_lot', 'floors','view', 'condition', 'yr_renovated', 'zipcode', 'long','sqft_living15','sqft_lot15','sqft_living'], axis=1)

In [ ]:
# Making predictions
y_pred_m2 = lm_2.predict(X_test_m2)

# Model Evaluation

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test, y_pred_m2)
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
# Plotting the error terms to understand the distribution.
fig = plt.figure()
sns.distplot((y_test-y_pred_m2),bins=50)
fig.suptitle('Error Terms', fontsize=20)                  # Plot heading 
plt.xlabel('Residual', fontsize=18)                  # X-label
plt.ylabel('Index', fontsize=16)   

In [ ]:
from sklearn import metrics
print('RMSE :', np.sqrt(metrics.mean_squared_error(y_test, y_pred_m2)))

In [ ]:
#Lower values of RMSE indicate better fit